<a href="https://colab.research.google.com/github/riti1302/Cartoon-Emotion-Recognition/blob/master/Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip "/content/drive/My Drive/Emotion_recognition.zip" -d "./"

In [0]:
cd 'Emotion_recognition'

/content/Emotion_recognition


In [0]:
!pip3 install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 39kB/s 
     |████████████████████████████████| 3.8MB 46.8MB/s 
     |████████████████████████████████| 512kB 44.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=7d7ac417c73d9e8f1f1549c4a7d6a10920faa5aba0e4ce278fd0a452300d5712
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc

In [0]:
!sh train.sh



W0415 06:55:19.875270 139999072065408 module_wrapper.py:139] From retrain.py:805: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.


W0415 06:55:19.875519 139999072065408 module_wrapper.py:139] From retrain.py:807: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

>> Downloading inception-2015-12-05.tgz 100.0%
Successfully downloaded inception-2015-12-05.tgz 88931400 bytes.
Instructions for updating:
Use tf.gfile.GFile.
W0415 06:55:21.923820 139999072065408 deprecation.py:323] From retrain.py:262: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.

W0415 06:55:21.924152 139999072065408 module_wrapper.py:139] From retrain.py:263: The name tf.GraphDef is deprecated. Please use tf.compat.v1.GraphDef instead.

2020-04-15 06:55:22.260277: W tensorflow/core/framework/op_def_util.cc:357] Op BatchNormWithGlobalNormalizat

In [0]:
%cd '../'
!zip -r Emotion_recognition.zip Emotion_recognition

/content
  adding: Emotion_recognition/ (stored 0%)
  adding: Emotion_recognition/get_dataset.py (deflated 47%)
  adding: Emotion_recognition/train.sh (deflated 49%)
  adding: Emotion_recognition/tf_files/ (stored 0%)
  adding: Emotion_recognition/tf_files/bottlenecks/ (stored 0%)
  adding: Emotion_recognition/tf_files/bottlenecks/sad/ (stored 0%)
  adding: Emotion_recognition/tf_files/bottlenecks/sad/frame139.jpg.txt (deflated 55%)
  adding: Emotion_recognition/tf_files/bottlenecks/sad/frame18.jpg.txt (deflated 55%)
  adding: Emotion_recognition/tf_files/bottlenecks/sad/frame48.jpg.txt (deflated 55%)
  adding: Emotion_recognition/tf_files/bottlenecks/sad/frame125.jpg.txt (deflated 55%)
  adding: Emotion_recognition/tf_files/bottlenecks/sad/frame127.jpg.txt (deflated 55%)
  adding: Emotion_recognition/tf_files/bottlenecks/sad/frame251.jpg.txt (deflated 55%)
  adding: Emotion_recognition/tf_files/bottlenecks/sad/frame174.jpg.txt (deflated 55%)
  adding: Emotion_recognition/tf_files/bott

In [0]:
!python classify.py 'test.jpg'
#cd 'Emotion_recognition'

/content/Emotion_recognition


In [0]:
import tensorflow as tf
import pandas as pd
import sys
import os
import cv2


# speicherorte fuer trainierten graph und labels in train.sh festlegen ##

def predict(image_data, sess, softmax_tensor):
    # holt labels aus file in array
    label_lines = [line.rstrip() for line in tf.gfile.GFile("tf_files/retrained_labels.txt")]
    predictions = sess.run(softmax_tensor, \
              {'DecodeJpeg/contents:0': image_data})
    # gibt prediction values in array zuerueck:

    top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
    # sortierung; circle -> 0, plus -> 1, square -> 2, triangle -> 3; array return bsp [3 1 2 0] -> sortiert nach groesster uebereinstimmmung

    # output
    emotion = label_lines[top_k[0]]
    score = predictions[0][top_k[0]]
    #print('%s (score = %.5f)' % (emotion, score))

    return (emotion, score)

def main():
    IMAGES_PATH = '/content/drive/My Drive/test_frames/'
    output = pd.DataFrame(columns = ['Frame_ID', 'Emotion'])

    df = pd.read_csv('../Test_annotated.csv')
    images_list = os.listdir(IMAGES_PATH)
    unknown_class_images = list(set(images_list) - set(df['Frame_ID']))
    print(unknown_class_images)

    for image_id, img_name in enumerate(df.Frame_ID.unique()):
        image_df = df[df.Frame_ID == img_name]
        image = cv2.imread(IMAGES_PATH + img_name)
        max_score = 0
        for _, row in image_df.iterrows():
            face = image[int(row['y_min']):int(row['y_max']), int(row['x_min']):int(row['x_max'])]
            success, encoded_face = cv2.imencode('.png', face)
            byte_array = encoded_face.tobytes()
             # Disable tensorflow compilation warnings
            os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
            
            # !! labels befinden sich jeweils in eigenen lines -> keine aenderung in retrain.py noetig -> falsche darstellung im windows editor !!

            # graph einlesen, wurde in train.sh -> call retrain.py trainiert
            with tf.gfile.FastGFile("tf_files/retrained_graph.pb", 'rb') as f:

                graph_def = tf.GraphDef()	## The graph-graph_def is a saved copy of a TensorFlow graph; objektinitialisierung
                graph_def.ParseFromString(f.read())	#Parse serialized protocol buffer data into variable
                _ = tf.import_graph_def(graph_def, name='')	# import a serialized TensorFlow GraphDef protocol buffer, extract objects in the GraphDef as tf.Tensor

              #https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/inception.py ; ab zeile 276

            with tf.Session() as sess:

                softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
              # return: Tensor("final_result:0", shape=(?, 4), dtype=float32); stringname definiert in retrain.py, zeile 1064

                emotion, score = predict(byte_array, sess, softmax_tensor)
            if score > max_score:
              max_score = score
              emotion_class = emotion
        index = int(''.join(list(filter(str.isdigit, img_name))))
        output.loc[index]=[img_name, emotion_class]
        print(img_name, emotion_class)

    for img_name in unknown_class_images:
         index = int(''.join(list(filter(str.isdigit, img_name))))
         output.loc[index]=[img_name, 'Unknown']
         print(img_name, 'Unknown')

    output = output.sort_index(axis = 0)
    output.to_csv('Test.csv', header=True, index=None)
#if __name__ == '__main__':
main()

['test7.jpg', 'test179.jpg', 'test144.jpg', 'test57.jpg', 'test117.jpg', 'test79.jpg', 'test37.jpg', 'test47.jpg', 'test67.jpg', 'test6.jpg', 'test136.jpg', 'test145.jpg', 'test122.jpg', 'test119.jpg', 'test101.jpg', 'test148.jpg', 'test133.jpg', 'test135.jpg', 'test123.jpg', 'test102.jpg', 'test104.jpg', 'test137.jpg', 'test107.jpg', 'test100.jpg', 'test103.jpg', 'test78.jpg', 'test35.jpg', 'test168.jpg', 'test28.jpg', 'test118.jpg', 'test25.jpg', 'test38.jpg', 'test134.jpg', 'test131.jpg', 'test71.jpg', 'test115.jpg', 'test40.jpg', 'test48.jpg', 'test5.jpg', 'test166.jpg', 'test31.jpg', 'test41.jpg', 'test39.jpg', 'test132.jpg', 'test139.jpg']
Instructions for updating:
Use tf.gfile.GFile.
test0.jpg happy
test1.jpg angry
test2.jpg surprised
test3.jpg sad
test4.jpg angry
test8.jpg sad
test9.jpg sad
test10.jpg happy
test11.jpg happy
test12.jpg happy
test13.jpg angry
test14.jpg happy
test15.jpg happy
test16.jpg happy
test17.jpg surprised
test18.jpg happy
test19.jpg surprised
test20.jpg 

In [0]:
cd Emotion_reconition_colab

/content/Emotion_reconition_colab


In [0]:
df = pd.DataFrame(columns=['name', 'emotion'])
df

,name,emotion


In [0]:
df.loc[7]=['ritika', 'sad']
df.loc[4]=['jatin', 'happy']
df.loc[0]=['kavita', 'angry']
df

,name,emotion
7,ritika,sad
4,jatin,happy
0,kavita,angry


In [0]:
df = df.sort_index(axis = 0)
df.to_csv('../test.csv', header=True, index = False)
df

,name,emotion
0,kavita,angry
4,jatin,happy
7,ritika,sad


In [0]:
str1 = 'test34.jpg'
num = int(''.join(list(filter(str.isdigit, str1))))
num

34